https://github.com/DataTalksClub/machine-learning-zoomcamp/blob/master/cohorts/2025/03-classification/homework.md

In [1]:
import numpy as np
import pandas as pd

In [2]:
from IPython.display import display

In [3]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import mutual_info_score
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

### Read the data

In [4]:
#!wget -P data https://raw.githubusercontent.com/alexeygrigorev/datasets/master/course_lead_scoring.csv

In [5]:
df = pd.read_csv('data/course_lead_scoring.csv')
print(df.shape)
df.head()

(1462, 9)


,lead_source,industry,number_of_courses_viewed,annual_income,employment_status,location,interaction_count,lead_score,converted
0,paid_ads,NaN,1,79450.0,unemployed,south_america,4,0.94,1
1,social_media,retail,1,46992.0,employed,south_america,1,0.80,0
2,events,healthcare,5,78796.0,unemployed,australia,3,0.69,1
3,paid_ads,retail,2,83843.0,NaN,australia,1,0.87,0
4,referral,education,3,85012.0,self_employed,europe,3,0.62,1


In this dataset our desired target for classification task will be `converted` variable - has the client signed up to the platform or not.

### 2. Prepare the data

Check if the missing values are presented in the features.

In [6]:
num_columns = list(df.select_dtypes('number').columns)[:-1]
print(num_columns)
df[num_columns].isna().sum()

['number_of_courses_viewed', 'annual_income', 'interaction_count', 'lead_score']


number_of_courses_viewed      0
annual_income               181
interaction_count             0
lead_score                    0
dtype: int64

In [7]:
# For numerical features, replace with with 0.0
df[num_columns] = df[num_columns].fillna(0)

In [8]:
cat_columns = list(df.select_dtypes('object').columns)
print(cat_columns)
df[cat_columns].isna().sum()

['lead_source', 'industry', 'employment_status', 'location']


lead_source          128
industry             134
employment_status    100
location              63
dtype: int64

In [9]:
# For caterogiral features, replace them with 'NA'
df[cat_columns] = df[cat_columns].fillna('NA')

In [10]:
target = 'converted'

### Question 1

What is the most frequent observation (mode) for the column `industry`?

In [11]:
df['industry'].mode()[0]

'retail'

Ans: retail

### Question 2

Create the correlation matrix for the numerical features of your dataset. In a correlation matrix, you compute the correlation coefficient between every pair of features.

In [12]:
df_corr = df[num_columns].corr()
df_corr.style.background_gradient()

,number_of_courses_viewed,annual_income,interaction_count,lead_score
number_of_courses_viewed,1.000000,0.009770,-0.023565,-0.004879
annual_income,0.009770,1.000000,0.027036,0.015610
interaction_count,-0.023565,0.027036,1.000000,0.009888
lead_score,-0.004879,0.015610,0.009888,1.000000


In [13]:
scores = []
for col1, col2 in [
    ('interaction_count','lead_score'),
    ('number_of_courses_viewed','lead_score'),
    ('number_of_courses_viewed','interaction_count'),
    ('annual_income','interaction_count')
]:
    scores.append([col1, col2, abs(df_corr[col1][col2])])

df_scores = pd.DataFrame(scores,columns=['col1','col2','corr'])
df_scores.sort_values(by='corr', ascending=False, inplace=True)
df_scores.head(1)

,col1,col2,corr
3,annual_income,interaction_count,0.027036


Ans: `annual_income` and `interaction_count` has the biggest correlation

### 3. Split the data

* Split your data in train/val/test sets with 60%/20%/20% distribution.
* Use Scikit-Learn for that (the train_test_split function) and set the seed to 42.
* Make sure that the target value y is not in your dataframe.

In [14]:
df_train_full, df_test = train_test_split(df, test_size=0.2, random_state=42)
df_train, df_val = train_test_split(df_train_full, test_size=0.25, random_state=42)
df_train.shape, df_val.shape, df_test.shape

((876, 9), (293, 9), (293, 9))

In [15]:
df_train.reset_index(drop=True, inplace=True)
df_val.reset_index(drop=True, inplace=True)
df_test.reset_index(drop=True, inplace=True)

y_train = df_train.pop(target)
y_val = df_val.pop(target)
y_test = df_test.pop(target)

### Question 3

* Calculate the mutual information score between y and other categorical variables in the dataset. Use the training set only.
* Round the scores to 2 decimals using round(score, 2).

First, look at categorical columns

In [16]:
global_target = df['converted'].mean()
global_target

np.float64(0.6190150478796169)

In [17]:
for col in cat_columns:
    print('>>>',col)
    df_agg = df.groupby(col)[target].agg(['mean','count'])
    df_agg['diff'] = df_agg['mean'] - global_target
    df_agg['ratio'] = df_agg['mean'] / global_target
    display(df_agg)
    print()

>>> lead_source


,mean,count,diff,ratio
lead_source,,,,
NA,0.671875,128,0.052860,1.085394
events,0.596000,250,-0.023015,0.962820
organic_search,0.617021,282,-0.001994,0.996779
paid_ads,0.446970,264,-0.172045,0.722066
referral,0.807692,260,0.188677,1.304802
social_media,0.604317,278,-0.014699,0.976255



>>> industry


,mean,count,diff,ratio
industry,,,,
NA,0.559701,134,-0.059314,0.904181
education,0.748663,187,0.129648,1.209442
finance,0.595000,200,-0.024015,0.961204
healthcare,0.604278,187,-0.014737,0.976193
manufacturing,0.666667,174,0.047652,1.076980
other,0.611111,198,-0.007904,0.987231
retail,0.586207,203,-0.032808,0.946999
technology,0.569832,179,-0.049183,0.920547



>>> employment_status


,mean,count,diff,ratio
employment_status,,,,
NA,0.560000,100,-0.059015,0.904663
employed,0.689024,328,0.070009,1.113098
self_employed,0.653409,352,0.034394,1.055563
student,0.652299,348,0.033284,1.053769
unemployed,0.497006,334,-0.122009,0.802898



>>> location


,mean,count,diff,ratio
location,,,,
NA,0.698413,63,0.079398,1.128264
africa,0.601064,188,-0.017951,0.971000
asia,0.620513,195,0.001498,1.002420
australia,0.605405,185,-0.013610,0.978014
europe,0.652778,216,0.033763,1.054543
middle_east,0.631313,198,0.012298,1.019867
north_america,0.595556,225,-0.023459,0.962102
south_america,0.598958,192,-0.020057,0.967599


In [18]:
scores = []
for col in cat_columns:
    scores.append([col, mutual_info_score(df_train[col], y_train)])
df_scores = pd.DataFrame(scores, columns=['column','mutual_score'])
df_scores.sort_values('mutual_score', ascending=False)

,column,mutual_score
0,lead_source,0.035396
2,employment_status,0.012938
1,industry,0.011575
3,location,0.004464


Ans: `lead_score` has the biggest mutual information score

### Question 4

* Now let's train a logistic regression.
* Remember that we have several categorical variables in the dataset. Include them using one-hot encoding.
* Fit the model on the training dataset.
    - To make sure the results are reproducible across different versions of Scikit-Learn, fit the model with these parameters:
    - model = LogisticRegression(solver='liblinear', C=1.0, max_iter=1000, random_state=42)
* Calculate the accuracy on the validation dataset and round it to 2 decimal digits.

In [19]:
def train_model(df_train, y_train, C=1.0):
    # one-hot encoding
    dv = DictVectorizer(sparse=False)
    train_dicts = df_train.to_dict(orient='records')
    X_train = dv.fit_transform(train_dicts)

    model = LogisticRegression(solver='liblinear', C=C, max_iter=1000, random_state=42)
    model.fit(X_train, y_train)

    return dv, model    

In [20]:
def evaluate_model(dv, model, df_pred, y_true):
    
    pred_dicts = df_pred.to_dict(orient='records')
    X_pred = dv.transform(pred_dicts)
    y_pred = model.predict(X_pred)    

    accuracy = (y_pred == y_true).mean()
    
    return accuracy, y_pred
    

In [21]:
dv, model = train_model(df_train, y_train)

In [22]:
global_accuracy, _ = evaluate_model(dv, model, df_val, y_val)
global_accuracy

np.float64(0.6996587030716723)

Ans: 0.7

### Question 5

* Let's find the least useful feature using the feature elimination technique.
* Train a model using the same features and parameters as in Q4 (without rounding).
* Now exclude each feature from this set and train a model without it. Record the accuracy for each model.
* For each feature, calculate the difference between the original accuracy and the accuracy without the feature.

In [23]:
results = []
for col in cat_columns:
    dv, model = train_model(df_train.drop(col, axis=1), y_train)
    accuracy, _ = evaluate_model(dv, model, df_val.drop(col, axis=1), y_val)
    results.append([col, accuracy])
df_scores = pd.DataFrame(results, columns=['column','accuracy'])
df_scores['diff'] = abs(global_accuracy - df_scores['accuracy'])
df_scores.sort_values('diff', inplace=True)
df_scores
    

,column,accuracy,diff
1,industry,0.699659,0.000000
2,employment_status,0.696246,0.003413
0,lead_source,0.703072,0.003413
3,location,0.709898,0.010239


Ans: `industry` has the smallest difference

### Question 6

* Now let's train a regularized logistic regression.
* Let's try the following values of the parameter C: [0.01, 0.1, 1, 10, 100].
* Train models using all the features as in Q4.
* Calculate the accuracy on the validation dataset and round it to 3 decimal digits.

In [24]:
results = []
for C in [0.01, 0.1, 1, 10, 100]:
    dv, model = train_model(df_train, y_train, C)
    accuracy, _ = evaluate_model(dv, model, df_val, y_val)
    results.append([C, accuracy])
df_scores = pd.DataFrame(results, columns=['C','accuracy'])
df_scores.sort_values(['accuracy', 'C'], ascending=[False,True], inplace=True)
df_scores

,C,accuracy
0,0.01,0.699659
1,0.10,0.699659
2,1.00,0.699659
3,10.00,0.699659
4,100.00,0.699659


Ans: C = 0.01